In [ ]:
class Repetition(Shapefile):
    def __init__(self, datasetNumber, inputFolder, outputFolder):
        super().__init__(datasetNumber,inputFolder,outputFolder)



    def remove_repetitions_from_components_points(self, comments=True):
        
        removed_counts = {}
        for component, gdf in self.components_points_gdf.items():

            all_att = gdf.columns.tolist()
            all_att.remove(self.idColumunNodes)

            ids_before = gdf[self.idColumunNodes].tolist()
            #print(ids_before)

            '''if self.idColumunNodes in gdf.columns:
                gdf.drop(columns=self.idColumunNodes, inplace=True)
                if comments:
                    print(f"The id column '{self.idColumunNodes}' is removed from ", component)
            if "id" in gdf.columns:
                gdf.drop(columns="id", inplace=True)
                if comments:
                    print(f"The id column  id is removed from ", component)'''

            # Get the initial number of rows
            initial_count = len(gdf)

            # Remove duplicate rows based on all columns and update the dictionary
            self.components_points_gdf[component] = gdf.drop_duplicates(all_att)

            ids_after = self.components_points_gdf[component][self.idColumunNodes].tolist()
            #print(ids_after)

            difference_ids= list(set(ids_before)-set(ids_after))
            print(f"repeated ids {difference_ids}")

            # Calculate the number of removed rows
            removed_count = initial_count - len(self.components_points_gdf[component])
            removed_counts[component] = removed_count

            if comments:
                print(f"The component {component} is saved in the folder {self.output_folder}")
            componentPath = "Data SIG/" + self.dataset + "/"+self.output_folder+"/"+component+".shp"  # Path to the nodes shapefile
            self.components_points_gdf[component].to_file(componentPath, driver='ESRI Shapefile')

        if comments:
            table = PrettyTable()
            table.field_names = ["Component", "Removed Rows"]
            for component, count in removed_counts.items():
                table.add_row([component, count])
            print("Removed Duplicate Rows:")
            print(table)

    def remove_repetitions_pipes(self, comments = False):
        self.pipesPath = "Data SIG/" + self.dataset + "/"+self.input_folder+"/Pipes.shp"
        self.read_shapefile("Pipes")

        all_att = self.gdfPipes.columns.tolist()
        all_att.remove(self.idColumunPipes)

        ids_before =  self.gdfPipes[self.idColumunPipes].tolist()

        # Get the initial number of rows
        '''if 'id' in self.gdfPipes.columns:
            self.gdfPipes.drop(columns='id', inplace=True)
            if comments:
                    print(f"The id column id is removed from Pipes")
        if self.idColumunPipes in self.gdfPipes.columns:
            self.gdfPipes.drop(columns=self.idColumunPipes, inplace=True)
            if comments:
                    print(f"The id column '{self.idColumunNodes}' is removed from Pipes")'''
            
        initial_count = len(self.gdfPipes)
        
        # Remove duplicate rows based on all columns and update the dictionary
        self.gdfPipes = self.gdfPipes.drop_duplicates(all_att)

        ids_after = self.gdfPipes[self.idColumunPipes].tolist()
        #print(ids_after)

        difference_ids= list(set(ids_before)-set(ids_after))
        print(f"repeated ids {difference_ids}")
        
        # Calculate the number of removed rows
        removed_count = initial_count - len(self.gdfPipes)
        if comments:
            print(f'The number of removed pipes is {removed_count}')
            
        path_saved = "Data SIG/" + self.dataset + "/"+self.output_folder+"/Pipes.shp"
        self.gdfPipes.to_file(path_saved, driver='ESRI Shapefile')

        if comments:
            print(f'the pipe databse is saved in {path_saved}')
    
    def remove_repetitions_nodes_based_on_geometry_combined_file(self, comments=True):
        self.nodesPath = "Data SIG/" + self.dataset + "/"+self.input_folder+"/Nodes.shp"
        self.read_shapefile("Nodes")
        initial_count = len(self.gdfNodes)
        # Remove duplicate rows based on geometry and update the dictionary
        self.gdfNodes =self.gdfNodes.drop_duplicates(subset=['geometry'])

        
        removed_count = initial_count - len(self.gdfNodes)

        if comments:
            print(f'The number of removed pipes is {removed_count}')
        
        self.add_unique_id_attribute("Nodes")
            
        path_saved = "Data SIG/" + self.dataset + "/"+self.output_folder+"/Nodes.shp"
        self.gdfNodes.to_file(path_saved, driver='ESRI Shapefile')

        if comments:
            print(f'The pipe databse is saved in {path_saved}')

    def repeated_pipes_by_position(self, comments=False):
        self.nodesPath = "Data SIG/" + self.dataset + "/"+self.input_folder+"/Pipes.shp"
        self.read_shapefile("Pipes")
        self.add_unique_id_attribute("Pipes")
        # Convert geometry to WKT string for comparison
        self.gdfPipes['geometry_wkt'] = self.gdfPipes['geometry'].apply(
            lambda geom: geom.wkt if geom is not None else None
        )
        ids_before =  self.gdfPipes[self.idColumunPipes].tolist()

        # Group pipes by geometry and count occurrences
        grouped = self.gdfPipes.groupby('geometry_wkt').size()

        # Filter groups with count > 1
        filtered_groups = grouped[grouped > 1]

        # Get the geometries of filtered groups
        filtered_geometries_wkt = filtered_groups.index

        # Create a dictionary to store categories and their corresponding IDs
        categories = {}

        totalLength = 0

        # Iterate through filtered geometries and populate the dictionary
        for geometry_wkt in filtered_geometries_wkt:
            if geometry_wkt is None:
                continue
            ids = self.gdfPipes[self.gdfPipes['geometry_wkt'] == geometry_wkt]['id'].tolist()
            categories[geometry_wkt] = ids

            totalLength = totalLength + len(categories[geometry_wkt])

        ids_after = self.gdfPipes[self.idColumunPipes].tolist()
        #print(ids_after)

        difference_ids= list(set(ids_before)-set(ids_after))
        print(f"repeated ids {difference_ids}")

        # Print the categories and their IDs
        print("The number of elements that will be removed :", totalLength - len(categories))

        if comments:
            for category_wkt, ids in categories.items():
                print(f"Category: {category_wkt} | IDs: {ids}")

        # Drop the temporary column
        self.gdfPipes.drop(columns=['geometry_wkt'], inplace=True)

    def repeated_nodes_by_position(self, comments = False):
        self.nodesPath = "Data SIG/" + self.dataset + "/"+self.input_folder+"/Nodes.shp"
        self.read_shapefile("Nodes")
        self.add_unique_id_attribute("Nodes")
        # Convert geometry to WKT string for comparison
        self.gdfNodes['geometry_wkt'] = self.gdfNodes['geometry'].apply(
            lambda geom: geom.wkt if geom is not None else None
        )

        # Group nodes by geometry and count occurrences
        grouped = self.gdfNodes.groupby('geometry_wkt').size()

        # Filter groups with count > 1
        filtered_groups = grouped[grouped > 1]

        # Get the geometries of filtered groups
        filtered_geometries_wkt = filtered_groups.index

        # Create a dictionary to store categories and their corresponding IDs
        categories = {}
        totalLength = 0

        # Iterate through filtered geometries and populate the dictionary
        for geometry_wkt in filtered_geometries_wkt:
            if geometry_wkt is None:
                continue
            ids = self.gdfNodes[self.gdfNodes['geometry_wkt'] == geometry_wkt]['id'].tolist()
            categories[geometry_wkt] = ids
            totalLength = totalLength + len(categories[geometry_wkt])

        print("The number of elements that will be removed :", totalLength - len(categories))

        # Print the categories and their IDs
        if comments:
            for category_wkt, ids in categories.items():
                print(f"Category: {category_wkt} | IDs: {ids}")

        # Drop the temporary column
        self.gdfNodes.drop(columns=['geometry_wkt'], inplace=True)

    def remove_repetitions_pipes_based_on_geometry(self, comments=True):
        
        self.pipesPath = "Data SIG/" + self.dataset + "/"+self.input_folder+"/Pipes.shp"
        self.read_shapefile("Pipes")

        ids_before =  self.gdfPipes[self.idColumunPipes].tolist()

        initial_count = len(self.gdfPipes)
        self.gdfPipes = self.gdfPipes.drop_duplicates(subset=['geometry'])

        ids_after = self.gdfPipes[self.idColumunPipes].tolist()
        #print(ids_after)

        difference_ids= list(set(ids_before)-set(ids_after))
        print(f"repeated ids {difference_ids}")

        numberRemoved = initial_count - len(self.gdfPipes)

        if comments:
            print(f'The number of removed pipes is {numberRemoved}')

        path_saved = "Data SIG/" + self.dataset + "/"+self.output_folder+"/Pipes.shp"
        self.gdfPipes.to_file(path_saved, driver='ESRI Shapefile')

        if comments:
            print(f'The pipe databse is saved in {path_saved}')
    
    def remove_repetitions_nodes_based_on_geometry_components(self, comments=True):
        removed_counts = {}
        for component, gdf in self.components_points_gdf.items():
            # Get the initial number of rows
            initial_count = len(gdf)
            ids_before = gdf[self.idColumunNodes].tolist()

            # Remove duplicate rows based on geometry and update the dictionary
            self.components_points_gdf[component] = gdf.drop_duplicates(subset=['geometry'])

            # Calculate the number of removed rows
            removed_count = initial_count - len(self.components_points_gdf[component])
            removed_counts[component] = removed_count

            ids_after = self.components_points_gdf[component][self.idColumunNodes].tolist()
            #print(ids_after)

            difference_ids= list(set(ids_before)-set(ids_after))
            print(f"repeated ids {difference_ids}")

            if comments:
                print(f"The component {component} is saved in the folder {self.output_folder}")
                componentPath = "Data SIG/" + self.dataset + "/"+self.output_folder+"/"+component+".shp"  # Path to the nodes shapefile
                self.components_points_gdf[component].to_file(componentPath, driver='ESRI Shapefile')


        if comments:
            table = PrettyTable()
            table.field_names = ["Component", "Removed Rows"]
            for component, count in removed_counts.items():
                table.add_row([component, count])
            print("Removed Duplicate Rows Based on Geometry:")
            print(table)
